In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import math
from scipy.stats import entropy
from numpy.linalg import norm
from scipy.integrate import solve_ivp
from joblib import Parallel, delayed
import concurrent.futures
from numba import jit
import itertools;

/Users/jakeschaefer/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#Bifurcation Plots with z on the y axis and r on the x axis
def lorenz_system(x, y, z, r=28, b=8/3, s=10):
    x_dot = s * (y - x)
    y_dot = r * x - y - x * z
    z_dot = x * y - b * z
    return x_dot, y_dot, z_dot


ds = 0.1  # parameter step size
s = np.arange(25, 250, ds)  # parameter range
dt = 0.001  # time step
t = np.arange(0, 10, dt)  # time range
b_range = np.arange(4/3, 12/3, 0.1)


# initialize solution arrays
xs = np.empty(len(t) + 1)
ys = np.empty(len(t) + 1)
zs = np.empty(len(t) + 1)

# initial values x0,y0,z0 for the system
xs[0], ys[0], zs[0] = (1, 1, 1)


# Save the plot points coordinates and plot the with a single call to plt.plot
# instead of plotting them one at a time, as it's much more efficient
r_maxes = []
z_maxes = []
r_mins = []
z_mins = []

#save the r values for lyapunov plot
rvalues = []
for b in b_range:
    for sigma in s:
        rvalues.append(sigma)
        # Print something to show everything is running
        # print(f"{R=:.2f}")
        for i in range(len(t)):
            # approximate numerical solutions to system
            x_dot, y_dot, z_dot = lorenz_system(xs[i], ys[i], zs[i], s=sigma)
            xs[i + 1] = xs[i] + (x_dot * dt)
            ys[i + 1] = ys[i] + (y_dot * dt)
            zs[i + 1] = zs[i] + (z_dot * dt)
        # calculate and save the peak values of the z solution
        for i in range(1, len(zs) - 1):
            # save the local maxima
            if zs[i - 1] < zs[i] and zs[i] > zs[i + 1]:
                r_maxes.append(s)
                z_maxes.append(zs[i])
            # save the local minima
            elif zs[i - 1] > zs[i] and zs[i] < zs[i + 1]:
                r_mins.append(s)
                z_mins.append(zs[i])

        # "use final values from one run as initial conditions for the next to stay near the attractor"
        xs[0], ys[0], zs[0] = xs[i], ys[i], zs[i]

print(np.array(r_maxes).shape, np.array(z_maxes).shape)
rvalues = np.array(rvalues)

#looking at how system changes as the parameter r changes - we are quantifying system change through the change in z
#print(r_maxes, z_maxes)
plt.scatter(r_maxes, z_maxes, color="black", s=0.5, alpha=0.2)
plt.scatter(r_mins, z_mins, color="red", s=0.5, alpha=0.2)

plt.title('Bifurcation plot for varying r in the Lorenz system')
plt.ylabel('z')
plt.xlabel('r')

#plt.xlim(0, 350)
#plt.ylim(0, 250)
plt.tick_params(labelsize = 15)
plt.show()

In [ ]:
@jit(nopython=True)
def func(t, v, sigma, r, b):
        x, y, z = v
        return [ sigma * (y - x), r * x - y - x * z, x * y - b * z ]

@jit(nopython=True)
def JM(t, v, sigma, r, b):
        x, y, z = v
        return np.array([[-sigma, sigma, 0], [r - z, -1, -x], [y, x, -b]])

    
#Function of simpler lyap
def eulerlorenzlyap (sigma, r, b):
    # parameters
    # sigma = 16
    # r = 45.92 
    # b = 4.0
    # ODE system
    
    # dimension of the system (to keep things general)
    n = 3

    # number of Lyapunov exponents
    n_lyap = 3

    # (dis)assemble state and tangent vectors (or their derivatives)
    # into one vector for integrator:
    assemble = lambda v, U: np.hstack((v,U.flatten()))
    disassemble = lambda state: (state[:n], state[n:].reshape(n,n_lyap))

    def func_with_lyaps(t, state, *pars):
        v, U = disassemble(state)
        dv = func(t, v, *pars)
        dU = JM(t, v, *pars) @ U
        return assemble(dv, dU)

    # initial states:
    v = np.random.random(n)
    U = np.random.random((n_lyap,n))

    lyaps = [] # local Lyapunov exponents

    dt = 1
    iters = 1000

    for _ in range(iters):
        sol = solve_ivp(
                func_with_lyaps,
                [0, dt],
                assemble(v,U),
                t_eval=[dt],
                args=(sigma, r, b),
                max_step=dt,
            )
        v,U = disassemble(sol.y.flatten())
        U,R = np.linalg.qr(U)
        lyaps.append(np.log(abs(R.diagonal()))/(dt*np.log(2)))

    transient_steps = 100
    # result:
    #print(*np.average(lyaps[transient_steps:],axis=0))
    exp = np.average(lyaps[transient_steps:],axis=0)
    maximal = exp.max()
    return maximal


def rk4lorenzlyap(sigma, rho, beta, initialcoord):
    #initalcoord = [x,y,z]
    # Evolution equation of tracjectories and tangential vectors
    def f(r):
        x = r[0]
        y = r[1]
        z = r[2]

        fx = sigma * (y - x)
        fy = x * (rho - z) - y
        fz = x * y - beta * z

        return np.array([fx,fy,fz], float)

    def jacobian(r):
        M = np.zeros([3,3])
        M[0,:] = [- sigma, sigma, 0]
        M[1,:] = [rho - r[2], -1, - r[0] ]
        M[2,:] = [r[1], r[0], -beta]

        return M

    def g(d, r):
        dx = d[0]
        dy = d[1]
        dz = d[2]
    
        M = jacobian(r)

        dfx = np.dot(M, dx)
        dfy = np.dot(M, dy)
        dfz = np.dot(M, dz)

        return np.array([dfx, dfy, dfz], float)

    # Initial conditions
    d = np.array([[1,0,0], [0,1,0], [0,0,1]], float) #starting derivative step size?
    r = np.array(initialcoord, float) #starting x, y, z, was 19, 20, 50
    #sigma, rho, beta = 16, 45.92, 4.0 

    T  = 10**5                         # time steps 
    dt = 0.01                          # time increment
    Teq = 10**4                        # Transient time

    l1, l2, l3 = 0, 0, 0               # Lengths

    xpoints, ypoints, zpoints  = [], [], []

    # Transient
    for t in range(Teq):
        # RK4 - Method 
        k1  = dt * f(r)                 
        k11 = dt * g(d, r)
    
        k2  = dt * f(r + 0.5 * k1)
        k22 = dt * g(d + 0.5 * k11, r + 0.5 * k1)

        k3  = dt * f(r + 0.5 * k2)
        k33 = dt * g(d + 0.5 * k22, r + 0.5 * k2)

        k4  = dt * f(r + k3)
        k44 = dt * g(d + k33, r + k3)

        r  += (k1  + 2 * k2  + 2 * k3  + k4)  / 6
        d  += (k11 + 2 * k22 + 2 * k33 + k44) / 6

        # Gram-Schmidt-Scheme
        orth_1 = d[0]                    
        d[0] = orth_1 / norm(orth_1)

        orth_2 = d[1] - np.dot(d[1], d[0]) * d[0]
        d[1] = orth_2 / norm(orth_2)

        orth_3 = d[2] - (np.dot(d[2], d[1]) * d[1]) - (np.dot(d[2], d[0]) * d[0]) 
        d[2] = orth_3 / norm(orth_3)

    for t in range(T):
        k1  = dt * f(r)                 
        k11 = dt * g(d, r)

        k2  = dt * f(r + 0.5 * k1)
        k22 = dt * g(d + 0.5 * k11, r + 0.5 * k1)
    
        k3  = dt * f(r + 0.5 * k2)
        k33 = dt * g(d + 0.5 * k22, r + 0.5 * k2)

        k4  = dt * f(r + k3)
        k44 = dt * g(d + k33, r + k3)

        r  += (k1  + 2 * k2  + 2 * k3  + k4)  / 6
        d  += (k11 + 2 * k22 + 2 * k33 + k44) / 6

        orth_1 = d[0]                    # Gram-Schmidt-Scheme
        l1 += np.log(norm(orth_1))
        d[0] = orth_1 / norm(orth_1)

        orth_2 = d[1] - np.dot(d[1], d[0]) * d[0]
        l2 += np.log(norm(orth_2))
        d[1] = orth_2 / norm(orth_2)

        orth_3 = d[2] - (np.dot(d[2], d[1]) * d[1]) - (np.dot(d[2], d[0]) * d[0]) 
        l3 += np.log(norm(orth_3))
        d[2] = orth_3 / norm(orth_3)


    lya1 = l1 / ((dt * T) * np.log(2))
    lya2 = l2 / (((dt * T)  - lya1)*np.log(2))
    lya3 = l3 / (((dt * T) -  lya1 - lya2)*np.log(2))
    
    exps = [lya1, lya2, lya3]
    exps = np.array(exps)
    print(exps)
    
    maximal = exps.max()
    return maximal




In [ ]:
ds = 0.1  # parameter step size
s_range = np.arange(1, 100, ds)  # parameter range
b_range = np.arange(4/3, 12/3, 0.01)
def compute_lyapexp(beta):
    return eulerlorenzlyap(24.8, 28, beta)

with concurrent.futures.ProcessPoolExecutor() as executor:
    lyapexp = list(executor.map(compute_lyapexp, b_range))

lyapexp = np.array(lyapexp)
"""lyapexp = []
initialc = [19, 20, 50] #for Runge-Kutta method
for sigma in s_range:
    lyapout = eulerlorenzlyap(sigma, 28, 8/3)
    lyapexp.append(lyapout)

lyapexp = np.array(lyapexp) #turn list into array for plotting"""

plt.figure()
plt.plot(b_range, lyapexp, 'b', markersize =0.1)
plt.title('Lyapunov Exponents Plot')
plt.ylabel('Maximal Lyapunov Exponent', fontsize = 15)
plt.xlabel('b', fontsize = 15)
plt.tick_params(labelsize = 15)

In [ ]:
#Ordinal Pattern Analysis and Permutation Entropy Calculation
import numpy as np 
import matplotlib.pyplot as plt
import itertools
def ordinal_distribution(data, dx=3, dy=1, taux=1, tauy=1, return_missing=False, tie_precision=None):
    """
    Applies the Bandt and Pompe\\ [#bandt_pompe]_ symbolization approach to obtain 
    a probability distribution of ordinal patterns (permutations) from data.
    
    Parameters
    ----------
    data : array 
           Array object in the format :math:`[x_{1}, x_{2}, x_{3}, \\ldots ,x_{n}]`
           or  :math:`[[x_{11}, x_{12}, x_{13}, \\ldots, x_{1m}],
           \\ldots, [x_{n1}, x_{n2}, x_{n3}, \\ldots, x_{nm}]]`.
    dx : int
         Embedding dimension (horizontal axis) (default: 3).
    dy : int
         Embedding dimension (vertical axis); it must be 1 for time series 
         (default: 1).
    taux : int
           Embedding delay (horizontal axis) (default: 1).
    tauy : int
           Embedding delay (vertical axis) (default: 1).
    return_missing: boolean
                    If `True`, it returns ordinal patterns not appearing in the 
                    symbolic sequence obtained from **data** are shown. If `False`,
                    these missing patterns (permutations) are omitted 
                    (default: `False`).
    tie_precision : int
                    If not `None`, **data** is rounded with `tie_precision`
                    number of decimals (default: `None`).
    Returns
    -------
     : tuple
       Tuple containing two arrays, one with the ordinal patterns occurring in data 
       and another with their corresponding probabilities.

    Examples
    --------
    >>> ordinal_distribution([4,7,9,10,6,11,3], dx=2)
    (array([[0, 1],
            [1, 0]]),
     array([0.66666667, 0.33333333]))
    >>>
    >>> ordinal_distribution([4,7,9,10,6,11,3], dx=3, return_missing=True)
    (array([[0, 1, 2],
            [1, 0, 2],
            [2, 0, 1],
            [0, 2, 1],
            [1, 2, 0],
            [2, 1, 0]]),
     array([0.4, 0.2, 0.4, 0. , 0. , 0. ]))
    >>>
    >>> ordinal_distribution([[1,2,1],[8,3,4],[6,7,5]], dx=2, dy=2)
    (array([[0, 1, 3, 2],
            [1, 0, 2, 3],
            [1, 2, 3, 0]]),
     array([0.5 , 0.25, 0.25]))
    >>>
    >>> ordinal_distribution([[1,2,1,4],[8,3,4,5],[6,7,5,6]], dx=2, dy=2, taux=2)
    (array([[0, 1, 3, 2],
            [0, 2, 1, 3],
            [1, 3, 2, 0]]),
     array([0.5 , 0.25, 0.25]))
    """
    try:
        ny, nx = np.shape(data)
        data   = np.array(data)
    except:
        nx     = np.shape(data)[0]
        ny     = 1
        data   = np.array([data])

    if tie_precision is not None:
        data = np.round(data, tie_precision)

    partitions = np.concatenate([[np.concatenate(data[j:j+dy*tauy:tauy,i:i+dx*taux:taux]) for i in range(nx-(dx-1)*taux)] 
            for j in range(ny-(dy-1)*tauy)])

    symbols = np.apply_along_axis(np.argsort, 1, partitions)
    symbols, symbols_count = np.unique(symbols, return_counts=True, axis=0)

    probabilities = symbols_count/len(partitions)

    if return_missing==False:
        return symbols, probabilities
    
    else:
        all_symbols   = list(map(list,list(itertools.permutations(np.arange(dx*dy)))))
        for i in range(np.math.factorial(dx)):
            if i < len(symbols) and np.array_equal(symbols[i], all_symbols[i]):
                skip=0
            elif i == len(symbols):
                symbols = np.append(symbols, [all_symbols[i]], axis=0).tolist
                probabilities = np.append(probabilities, 0).tolist
            else:
                symbols = np.insert(symbols, i, all_symbols[i], axis=0).tolist
                probabilities = np.insert(probabilities, i, 0).tolist
        
        return symbols, probabilities

    
outputs = ordinal_distribution(lyapexp)
symbols = outputs[0]
probabilities = outputs[1]

print(symbols)
print(probabilities)

#Calculating Permutation Entropy
PE = 0

for i in probabilities:
    PE-=(i*np.log(i))/np.log(6)

print(PE)

In [ ]:
#print(eulerlorenzlyap(16, 45.92, 4.0))
#print(rk4lorenzlyap(16, 45.92, 4.0, [19, 20, 50]))

#Creating a list of maximal Lyapunov exponents to plot over a set range of r
dr = 0.5  # parameter step size
r = np.arange(25, 250, dr)  # parameter range


lyapexp = []
initialc = [19, 20, 50] #for Runge-Kutta method
for R in r:
    lyapout = eulerlorenzlyap(10, R, 8/3)
    lyapexp.append(lyapout)

lyapexp = np.array(lyapexp) #turn list into array for plotting

plt.figure()
plt.plot(r, lyapexp, 'b', markersize =0.1)
plt.title('Lyapunov Exponents Plot')
plt.ylabel('Maximal Lyapunov Exponent', fontsize = 15)
plt.xlabel('r', fontsize = 15)
plt.tick_params(labelsize = 15)


In [ ]:
dr = 0.5  # parameter step size
r = np.arange(25, 250, dr)  # parameter range


lyapexp = []
initialc = [19, 20, 50] #for Runge-Kutta method
for R in r:
    lyapout = eulerlorenzlyap(10, R, 8/3)
    lyapexp.append(lyapout)

lyapexp = np.array(lyapexp) #turn list into array for plotting

plt.figure()
plt.plot(r, lyapexp, 'b', markersize =0.1)
plt.title('Lyapunov Exponents Plot')
plt.ylabel('Maximal Lyapunov Exponent', fontsize = 15)
plt.xlabel('r', fontsize = 15)
plt.tick_params(labelsize = 15)

In [ ]:
ds = 0.1  # parameter step size
s_range = np.arange(1, 100, ds)  # Prandtl number range
dt = 0.01  # time step
t = np.arange(0, 2, dt)  # time range
b_range = np.arange(4/3, 12/3, 0.01)
beta_list = []
sigma_list = []

entropies_x = np.zeros((len(s_range), len(b_range)))
entropies_y = np.zeros((len(s_range), len(b_range)))
entropies_z = np.zeros((len(s_range), len(b_range)))

b_val = 0
s_val = 0
for i, sigma in enumerate(s_range):
    for j, beta in enumerate(b_range):
        xs, ys, zs = [1.0], [1.0], [1.0]  # Initial conditions
        print(sigma,beta)
        # ODE integration loop (as before)
        for k in range(len(t) - 1):
            x_dot, y_dot, z_dot = lorenz_system(xs[-1], ys[-1], zs[-1], b=beta, s=sigma)
            xs.append(xs[-1] + x_dot * dt)
            ys.append(ys[-1] + y_dot * dt)
            zs.append(zs[-1] + z_dot * dt)
        

        # Compute histograms and entropies (as before)
        prob_x, _ = np.histogram(xs, bins=30, density=True)
        prob_y, _ = np.histogram(ys, bins=30, density=True)
        prob_z, _ = np.histogram(zs, bins=30, density=True)
        
        entropies_x[i, j] = entropy(prob_x, base=2)
        entropies_y[i, j] = entropy(prob_y, base=2)
        entropies_z[i, j] = entropy(prob_z, base=2)


total = np.array([entropies_x, entropies_y, entropies_z])
levels = np.linspace(np.min(total), np.max(total), 200)
X, Y = np.meshgrid(b_range, s_range)
ax = plt.figure().add_subplot()

cb = ax.contourf(X,Y, entropies_x)
ax.legend()
ax.set_title('Shannon Entropies in X vs Beta')
ax.set_xlabel('Beta')
ax.set_ylabel('Sigma')
plt.colorbar(cb)
plt.show()

ax = plt.figure().add_subplot()
cb = ax.contourf(X,Y, entropies_y)
ax.legend()
ax.set_title('Shannon Entropies in Y vs Beta')
ax.set_xlabel('Beta')
ax.set_ylabel('Sigma')
plt.colorbar(cb)
plt.show()

ax = plt.figure().add_subplot()
cb = ax.contourf(X,Y, entropies_z)
ax.legend()
ax.set_title('Shannon Entropies in Z vs Beta')
ax.set_xlabel('Beta')
ax.set_ylabel('Sigma')
plt.colorbar(cb)
plt.show()


"""
entropies_x = np.zeros(len(s_range))
entropies_y = np.zeros(len(s_range))
entropies_z = np.zeros(len(s_range))
for j, sigma in enumerate(s_range):
    xs, ys, zs = [1.0], [1.0], [1.0]  # Initial conditions
    #print(sigma,beta)
    # ODE integration loop (as before)
    for k in range(len(t) - 1):
        x_dot, y_dot, z_dot = lorenz_system(xs[-1], ys[-1], zs[-1], s=sigma)
        xs.append(xs[-1] + x_dot * dt)
        ys.append(ys[-1] + y_dot * dt)
        zs.append(zs[-1] + z_dot * dt)
    

    # Compute histograms and entropies (as before)
    prob_x, _ = np.histogram(xs, bins=30, density=True)
    prob_y, _ = np.histogram(ys, bins=30, density=True)
    prob_z, _ = np.histogram(zs, bins=30, density=True)
    
    entropies_x[j] = entropy(prob_x, base=2)
    entropies_y[j] = entropy(prob_y, base=2)
    entropies_z[j] = entropy(prob_z, base=2)
"""

"""
fig = plt.figure()
ax = plt.figure().add_subplot(projection='3d')
ax.contourf(s_range2.flatten(), sigma_list[:,0].flatten(), sigma_list[:,2].flatten())
#ax.plot(s_range2.flatten(), sigma_list[:,0].flatten(), sigma_list[:,2].flatten(), label = f'Lorenz Attractor for s={sigma:.1f}')
ax.legend()
plt.show()



fig = plt.figure()
ax = plt.figure().add_subplot(projection='3d')
ax.plot(s_range2.flatten(), sigma_list[:,0].flatten(), sigma_list[:,3].flatten(), label = f'Lorenz Attractor for s={sigma:.1f}')
ax.legend()
plt.show() 


"""
"""
# s_range, entropies_x, entropies_y, entropies_z = beta
plt.figure(figsize=(12, 4))
plt.subplot(131)
plt.plot(s_range, entropies_x)
plt.title('Shannon Entropy of X')
plt.xlabel('Sigma')
plt.ylabel('Entropy')

plt.subplot(132)
plt.plot(s_range, entropies_y)
plt.title('Shannon Entropy of Y')
plt.xlabel('Sigma')

plt.subplot(133)
plt.plot(s_range, entropies_z)
plt.title('Shannon Entropy of Z')
plt.xlabel('Sigma')

plt.tight_layout()
plt.show()"""